# 1. 텍스트 전처리

In [18]:
import pandas as pd # 데이터프레임을 다루는 기본 데이터 분석 라이브러리
import numpy as np # 수치 계산 및 배열 처리를 위한 라이브러리
import re # 정규표현식(re) 라이브러리
import nltk # 자연어처리용 통합 라이브러리
import inflect # 숫자 → 문자 변환 등을 위한 텍스트 수 처리 라이브러리
import contractions # 축약어 복원 (예: “don’t” → “do not”)

from bs4 import BeautifulSoup # HTML 태그 제거 등에 사용하는 웹 파싱 도구
from nltk import word_tokenize # 단어 토큰화 (문장을 단어로 나누는 작업) 도구
from nltk.corpus import stopwords # 의미 없는 단어(불용어)를 불러오기 위한 모듈 
from nltk.stem import WordNetLemmatizer # 단어의 원형(표제어)로 변환하는 데 사용
from nltk import pos_tag # 단어에 품사 태깅을 위한 모듈 (단어가 명사인지, 동사인지, 부사인지 판단)
from nltk.corpus import wordnet # WordNet 전용 품사 태그를 제공하는 모듈

from tqdm import tqdm # 작업 진행 상태를 시각적으로 표시하는 라이브러리
tqdm.pandas() # pandas에서 tqdm 진행률 바 사용 가능하게 확장

nltk.download('stopwords') # 불용어 리스트 다운로드
nltk.download('punkt') # 문장/단어 분리기용 데이터 다운로드
nltk.download('wordnet') # 표제어 처리용 WordNet 리소스 다운로드

stop_words = set(stopwords.words('english')) # 영어 불용어 목록 로드
lemmatizer = WordNetLemmatizer() # 표제어 추출을 위한 lemmatizer 객체 생성

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sinjeongho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sinjeongho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sinjeongho/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


텍스트 전처리 함수 정의

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'): return wordnet.ADJ # 형용사
    elif tag.startswith('V'): return wordnet.VERB # 동사
    elif tag.startswith('N'): return wordnet.NOUN # 명사
    elif tag.startswith('R'): return wordnet.ADV # 부사
    else: return wordnet.NOUN # 기본값은 명사로 설정

def clean_txt(txt): 
    if txt is not None: # 입력값이 None이 아닌 경우에만 처리
        txt = str(txt) # 문자열로 강제 변환
        txt = BeautifulSoup(txt, "html.parser").get_text() # HTML 태그 제거
        txt = contractions.fix(txt) # 축약어 복원 (e.g., "don't" → "do not")
        txt = re.sub('[^a-zA-Z]', ' ', txt) # 알파벳 제외한 문자 모두 공백으로 대체
        txt = txt.lower() # 모두 소문자로 변환
        txt = word_tokenize(txt) # 단어 단위로 토큰화
        txt = [word for word in txt if word not in stop_words] # 불용어 제거
        txt = [i for i in txt if len(i) > 2] # 길이가 2 이하인 단어 제거 (예: 'a', 'is' 등)
        txt = [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in pos_tag(txt)] # 표제어 추출 (품사 태깅 포함)
        txt = ' '.join(txt) # 다시 문자열로 결합 (공백 구분)
        return txt if txt else None # 결과가 빈 문자열일 경우 None 반환
    return None # 입력값이 None인 경우 None 반환

In [7]:
data= pd.read_pickle('final_use.pkl')
print(data.shape)
data.head()

(452947, 25)


,Unnamed: 0,review_res,review_res_url,rating,review_date,reviewer_url,review_text,photo,location,reputation,...,emotion,Social,Perception,motion,fake,experience,Psychological Distancing,review_extremity,style,preprocessing
0,0,Benemon,/biz/benemon-new-york?osq=Restaurants,5.0,2/16/2023,/user_details?userid=pkppXY0hRJbXt6ELI_9X9Q,Good Japanese street food.Great choice of sake...,3,0,6,...,6.67,6.67,6.67,0.00,0,11.0,0.00,25.0,0.00,"['good', 'japanese', 'street', 'food.great', '..."
1,1,Burger & Lobster - Flatiron NYC,/biz/burger-and-lobster-flatiron-nyc-new-york?...,5.0,1/29/2022,/user_details?userid=pkppXY0hRJbXt6ELI_9X9Q,Great service and food was awesome. Cool atmos...,6,0,6,...,6.67,13.33,26.67,13.33,0,11.0,0.00,25.0,13.33,"['great', 'service', 'food', 'awesome', 'cool'..."
2,2,Benemon,/biz/benemon-new-york?osq=Restaurants,5.0,1/25/2023,/user_details?userid=jH4neSuhaxEoPGBmpNNK5Q,I was really missing the amazing Izakayas in J...,3,0,268,...,5.26,4.68,11.11,2.92,0,50.0,2.92,25.0,1.17,"['really', 'miss', 'amaze', 'izakayas', 'japan..."
3,3,Saigon Shack,/biz/saigon-shack-new-york?osq=Restaurants,5.0,9/13/2021,/user_details?userid=jH4neSuhaxEoPGBmpNNK5Q,Cash only but there is an ATM right outside. E...,2,0,268,...,0.88,5.31,12.39,2.65,0,50.0,4.42,25.0,6.19,"['cash', 'right', 'outside', 'even', 'monday',..."
4,4,Thai Diner,/biz/thai-diner-new-york?osq=Restaurants,5.0,12/20/2022,/user_details?userid=jH4neSuhaxEoPGBmpNNK5Q,From the same team that brought us Uncle Boons...,1,0,268,...,1.73,7.79,7.36,0.87,0,50.0,3.89,25.0,1.73,"['team', 'bring', 'uncle', 'boons', 'favorite'..."


In [8]:
data.drop(['Unnamed: 0', 'review_res', 'review_res_url', 'rating', 'review_date',
       'reviewer_url',  'photo', 'location', 'reputation',
       'reviewer_photo', 'depth', 'text_structure', 'Cognition', 'Affect',
       'emotion', 'Social', 'Perception', 'motion',  'experience',
       'Psychological Distancing', 'review_extremity', 'style',
       'preprocessing'],axis=1,inplace=True)
data.head()

,review_text,fake
0,Good Japanese street food.Great choice of sake...,0
1,Great service and food was awesome. Cool atmos...,0
2,I was really missing the amazing Izakayas in J...,0
3,Cash only but there is an ATM right outside. E...,0
4,From the same team that brought us Uncle Boons...,0


In [9]:
df = pd.read_pickle('raw_data.pkl')
df['fake'].value_counts()

fake
0    409983
1     42964
Name: count, dtype: int64

## PreProcessing

In [10]:
def get_wordnet_pos(tag): #표제어 추출출`   `
    if tag.startswith('J'): return wordnet.ADJ # 형용사
    elif tag.startswith('V'): return wordnet.VERB # 동사
    elif tag.startswith('N'): return wordnet.NOUN # 명사
    elif tag.startswith('R'): return wordnet.ADV # 부사
    else: return wordnet.NOUN # 기본값은 명사로 설정

def clean_txt(txt): 
    if txt is not None: # 입력값이 None이 아닌 경우에만 처리
        txt = str(txt)
        if not re.search('[a-zA-Z]', txt):  # 영어가 없으면 제거
            return None
        txt = BeautifulSoup(txt, "html.parser").get_text() # HTML 태그 제거
        txt = contractions.fix(txt) # 축약어 복원 (e.g., "don't" → "do not")
        txt = re.sub('[^a-zA-Z]', ' ', txt) # 알파벳 제외한 문자 모두 공백으로 대체
        
        txt = txt.lower() # 모두 소문자로 변환
        txt = word_tokenize(txt) # 단어 단위로 토큰화
        txt = [word for word in txt if word not in stop_words] # 불용어 제거
        txt = [i for i in txt if len(i) > 2] # 길이가 2 이하인 단어 제거 (예: 'a', 'is' 등)
        txt = [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in pos_tag(txt)] # 표제어 추출 (품사 태깅 포함)
        txt = ' '.join(txt) # 다시 문자열로 결합 (공백 구분)
        return txt if txt else None # 결과가 빈 문자열일 경우 None 반환
    return None # 입력값이 None인 경우 None 반환

In [11]:
df['clean_text'] = df['review_text'].progress_apply(clean_txt)
print(df.shape)
df.head()

100%|██████████| 452947/452947 [08:35<00:00, 878.24it/s] 

(452947, 3)


,review_text,fake,clean_text
0,Good Japanese street food.Great choice of sake...,0,good japanese street food great choice sake qu...
1,Great service and food was awesome. Cool atmos...,0,great service food awesome cool atmosphere go ...
2,I was really missing the amazing Izakayas in J...,0,really miss amaze izakayas japan decide stop t...
3,Cash only but there is an ATM right outside. E...,0,cash atm right outside even monday night degre...
4,From the same team that brought us Uncle Boons...,0,team bring uncle boon old favorite michelin st...


In [13]:
df = df[df['clean_text'].notnull()]
df.isna().sum()

review_text    0
fake           0
clean_text     0
dtype: int64

In [14]:
df['fake'].value_counts()

fake
0    409977
1     42880
Name: count, dtype: int64

In [15]:
df['clean_text'].head

<bound method NDFrame.head of 0         good japanese street food great choice sake qu...
1         great service food awesome cool atmosphere go ...
2         really miss amaze izakayas japan decide stop t...
3         cash atm right outside even monday night degre...
4         team bring uncle boon old favorite michelin st...
                                ...                        
452942    best italian ever good food nice staff well lo...
452943                             cute place love food yum
452944    one bad meal recent memory food mediocre poorl...
452945    often cry amaze food laugh great service din p...
452946    food amaze arancini clam toast chitarra hot ch...
Name: clean_text, Length: 452857, dtype: object>

## Data Split

In [16]:
df_false = df[df['fake'] == 1].sample(42880)  # 긍정 리뷰 샘플링
df_true = df[df['fake'] == 0].sample(42880)  # 부정 리뷰 샘플링
# 데이터 결합
df_concat = pd.concat([df_false, df_true])
df_concat.reset_index(drop=True, inplace=True)  # 인덱스 초기화

In [19]:
df_train, df_test = train_test_split(df_concat, test_size=0.2, random_state=42, stratify=df_concat["fake"])

# 훈련 데이터 리뷰 텍스트와 라벨 추출
x_train = df_train['clean_text']
y_train = np.array(df_train['fake'])

# 테스트 데이터 리뷰 텍스트와 라벨 추출
x_test = df_test['clean_text']
y_test = np.array(df_test['fake'])

In [20]:
df_train.to_pickle('df_train.pkl')
df_test.to_pickle('df_test.pkl')